In [1]:
# 导入模块
import pandas as pd
import numpy as np

In [2]:
# 导入数据
filePath = './dataset/dialogue_content_20200724.csv'
pd.set_option('display.max_columns',100)# 设置显示数据的最大列数，防止出现省略号…，导致数据显示不全
pd.set_option('expand_frame_repr',False)# 当列太多时不自动换行
df = pd.read_csv(filePath, encoding="utf-8", dtype=str)

In [3]:
df.head(100)
len(df)

2194169

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2194169 entries, 0 to 2194168
Data columns (total 2 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   id       object
 1   content  object
dtypes: object(2)
memory usage: 33.5+ MB


In [5]:
df = df[df['content'].str.len() < 300]
tracks_no_duplicates = df.drop_duplicates('content')
# df[df.content].index
# df['content'].str.len()

In [6]:
len(tracks_no_duplicates)

1103166

In [7]:
from Sentiment_Inference import *

In [8]:
model = Sentiment_Analysis(max_seq_len=300, batch_size=2)

./sentiment_state_dict/sentiment.model.epoch.418 loaded!


In [9]:
# tracks_no_duplicates.drop(tracks_no_duplicates[len(tracks_no_duplicates.content) > 300].index, inplace=True)
print(tracks_no_duplicates)

              id                       content
0           5227                  你怎么变成异常了[流汗]
1           5785                            收到
2           6673                            宁宁
3           6674                     我今晚回去发给你哈
4           6676                          太尴尬了
...          ...                           ...
2194148  9185059                          我是十有
2194149  9185060                       我是Rose。
2194150  9185061  晕，包裹卡被我家大宝扔了[捂脸][捂脸][流泪][流泪]
2194155  9185087                        可以买礼包么
2194161  9185102                       下午吧[愉快]

[1103166 rows x 2 columns]


In [10]:
model(tracks_no_duplicates['content'][:100])


你怎么变成异常了[流汗]
负样本, 输出值0.00
----------
收到
负样本, 输出值0.02
----------
宁宁
负样本, 输出值0.01
----------
我今晚回去发给你哈
负样本, 输出值0.00
----------
太尴尬了
负样本, 输出值0.00
----------
宁宁 我想要个头图 类似于这种
负样本, 输出值0.00
----------
是的
负样本, 输出值0.41
----------
好的
正样本, 输出值1.00
----------
宁宁 你外套能给我穿下吗 我感觉我要吹感冒了.......
负样本, 输出值0.07
----------
宁宁 咱们这周推文改了 送优惠券 
负样本, 输出值0.22
----------
需要一个头图
负样本, 输出值0.00
----------
是的 
负样本, 输出值0.01
----------
看看能不能设计个 我也没有好的案例
负样本, 输出值0.00
----------
好的 你最棒啦
负样本, 输出值0.06
----------
我想下
负样本, 输出值0.00
----------
618第一波优惠来了！
负样本, 输出值0.00
----------
明天你来我有伴儿了
负样本, 输出值0.00
----------
每周六都是我自己
负样本, 输出值0.45
----------
对 我家漏水了
负样本, 输出值0.00
----------
哈哈哈哈
负样本, 输出值0.02
----------
那等详情我小改一下 你做你的今天 我做的以后再改吧
负样本, 输出值0.10
----------
改的改的
负样本, 输出值0.17
----------
这怎么看尺寸 就知道1:1
负样本, 输出值0.00
----------
1图：【宝宝馋了】让宝宝吃的更健康~
五种口味，多重营养


2图：香香脆脆，无添加剂，无盐少糖，遇水即化，方便携带
3图：营养成分表

4图；香精×  色素×   防腐剂× 添加剂x
5图：0.6cm小个头，锻炼宝宝抓取能力
正样本, 输出值0.60
----------
嗯、、、我想一下哈 618第二波福利来啦~
正样本, 输出值0.65
----------
然后背景跟字体上次那种换一下吧 怕顾客以为我们又发了一遍
负

In [11]:
model.result
pd.DataFrame(model.result, index=[0]).to_csv('./result.csv')

In [12]:
import paddlehub as hub

In [13]:
senta = hub.Module(name="senta_lstm")
test_text = ["这家餐厅很好吃", "这部电影真的很差劲"]
input_dict = {"text": test_text}
results = senta.sentiment_classify(data=input_dict)


[2020-07-25 14:44:53,615] [    INFO] - Installing senta_lstm module
[2020-07-25 14:44:53,780] [    INFO] - Module senta_lstm already installed in /Users/lei/.paddlehub/modules/senta_lstm
[2020-07-25 14:44:57,926] [    INFO] - Installing lac module
[2020-07-25 14:44:57,978] [    INFO] - Module lac already installed in /Users/lei/.paddlehub/modules/lac


In [14]:
for result in results:
    print(result['text'])
    print(result['sentiment_label'])
    print(result['sentiment_key'])
    print(result['positive_probs'])
    print(result['negative_probs'])

这家餐厅很好吃
1
positive
0.9285
0.0715
这部电影真的很差劲
0
negative
0.0187
0.9813


In [20]:
# print(tracks_no_duplicates['content'])
list = []
for each in tracks_no_duplicates['content'][:100]:
#     print(each)
    list.append(each)
input_dict1 = {"text": list}
senta.sentiment_classify(data=input_dict1)

[{'text': '你怎么变成异常了[流汗]',
  'sentiment_label': 0,
  'sentiment_key': 'negative',
  'positive_probs': 0.2046,
  'negative_probs': 0.7954},
 {'text': '收到',
  'sentiment_label': 1,
  'sentiment_key': 'positive',
  'positive_probs': 0.827,
  'negative_probs': 0.173},
 {'text': '宁宁',
  'sentiment_label': 1,
  'sentiment_key': 'positive',
  'positive_probs': 0.9136,
  'negative_probs': 0.0864},
 {'text': '我今晚回去发给你哈',
  'sentiment_label': 1,
  'sentiment_key': 'positive',
  'positive_probs': 0.9749,
  'negative_probs': 0.0251},
 {'text': '太尴尬了',
  'sentiment_label': 0,
  'sentiment_key': 'negative',
  'positive_probs': 0.0145,
  'negative_probs': 0.9855},
 {'text': '宁宁 我想要个头图 类似于这种',
  'sentiment_label': 1,
  'sentiment_key': 'positive',
  'positive_probs': 0.6245,
  'negative_probs': 0.3755},
 {'text': '是的',
  'sentiment_label': 1,
  'sentiment_key': 'positive',
  'positive_probs': 0.5669,
  'negative_probs': 0.4331},
 {'text': '好的',
  'sentiment_label': 1,
  'sentiment_key': 'positive',
  '